# Transito no Brasil, o que mudou em 10 anos?

Inspirado pela analise do Gabriel Gomes no LinkedIn (https://goo.gl/v1ZVdJ) e após ter conversado com o mesmo, decidi analisar o mesmo conjunto de dados, mas ao invés de dar uma ênfase no perfil das vítimas, decidir extrair as repostas para minhas própias perguntas, que são:

##### Infos sobre os veículos envolvidos:
	- Veículos com maior numero de vitimas fatais
	- Veículos com maior numero de vitimas ilesas
	- Veículos com maior índice de falha mecânica
	- Veículos em que se envolveram em acidente por estar acima da velocidade regulamentada (top alguma coisa)
	- Veículos com maior numero de capotamentos (Hilux, cadê vc...?)
	- O ano do veículo influencia no estado das vítimas? (Os carros mais novos são mais seguros?)

##### Infos sobre Dia, clima, época do ano
	- Dia da semana com maior numero de ocorrencias
	- Estado das vítimas x dia da semana
	- Numero de ocorrências x hora do dia
	- Épocas do ano mais propensas à acidentes
	- Condições climáticas mais perigosas
	- Existe alguma relação entre a condição meteorológica x fase do dia e tipo de acidente?

##### Infos sobre o local do acidente
	- Estradas com maior número de vítimas (ilesas, feridos leves, feridos graves, e mortos)
	- Municípios com maior número de acidentes
	- O tipo de pista (Duplicada, Mão dupla, etc.) e o traçado da via tem relação com o tipo de acidente?
	- Tipo de pista x traçado da via x número e estado das vítimas
	- Heatmap neles!

##### Infos sobre as vítimas
	- A idade está relacionada com o estado da vítima de acordo com o tipo do acidente (Idade x Estado da vitima x Tipo do acidente)
	- Numero de acidentes e vítimas por sexo do condutor

##### Infos sobre o evento 
	- Municípios com maior número de acidentes causados por ingestão de alcool
	- Tipo de acidente mais letal


Neste notebook irei tratar e explorar dados de 10 anos de registros (de 2008 á 2018) que foram consolidados em um arquivo .csv por processo de carga de dados em MS SQL Server e posterior exportação para arquivo .csv. Processo que pode ser acompanhado nestes links:

#### Importação dos dados pro SQL Server 
https://www.linkedin.com/pulse/transito-brasil-o-que-mudou-em-10-anos-parte-1-ramon-nogueira/

#### Exportação e pré processamento dos dados no Jupyter Lab
https://www.linkedin.com/pulse/transito-brasil-o-que-mudou-em-10-anos-parte-2-limpeza-nogueira/


### Sobre os dados
Os dados utilizados para esta análise estão disponíveis no site da PRF (https://www.prf.gov.br/portal/dados-abertos/acidentes/acidentes) na sessão 'Agrupados por Pessoa'.
Uma versão simples do dicionário destes dados está disponível no corpo do artigo : https://www.linkedin.com/pulse/transito-brasil-o-que-mudou-em-10-anos-parte-1-ramon-nogueira/

In [1]:
#Importação das bibliotecas utilizadas neste notebook

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

#Setando os plots para serem exibidos diretamente no notebook
%matplotlib inline 

In [2]:
#Importação dos dados.
#Detalhe para o atributo encoding que recebeu o parâmetro 'latin1' conforme setado na exportação dos dados do banco
#E o atributo low_memory que recebeu False como parâmetro para que os campos fossem lidos conforme consta no arquivo, de modo a evitar a tipagem de cada campo manualmente
rawdata = pd.read_csv('Acidentes_cons.csv', encoding='latin1', low_memory= False)

In [3]:
rawdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3453255 entries, 0 to 3453254
Data columns (total 38 columns):
reg_id                    int64
id                        object
pesid                     float64
data_inversa              object
dia_semana                object
horario                   object
uf                        object
br                        object
km                        object
municipio                 object
causa_acidente            object
tipo_acidente             object
classificacao_acidente    object
fase_dia                  object
sentido_via               object
condicao_metereologica    object
tipo_pista                object
tracado_via               object
uso_solo                  object
id_veiculo                object
tipo_veiculo              object
marca                     object
ano_fabricacao_veiculo    object
tipo_envolvido            object
estado_fisico             object
idade                     float64
sexo                      ob

Ok, temos os dados já carregados em memória e agora poderemos dar início ao processo de exploração e tratamento destes dados. Existem alguns tratamentos que devem ser aplicados a toda massa de dados desconhecida, estes serão destacados com um * na lista de procedimentos que desejo realizar antes de começar a fazer o 'feature engeneering' 

#### Limpeza dos dados:
    Checagem de valores nulos ou NaN * (de acordo com o dicionário que escrevi)
    Checagem de inconsistências em valores numéricos * (idade -1, etc)
    Checagem de inconsistências em valores de texto *
    Descarte de registros com muitos valores NaN *
    Limpeza dos valores de texto (se necessário)
    Tipagem correta dos campos (Números inteiros como inteiros, valores quebrados como float, etc)
    Exclusão de registros sem campos considerados chave, como município, causa_acidente, etc.
    Descarte de campos marcados para exclusão (de acordo com dicionário de dados)
    
Só depois de ter os dados limpos poderemos padronizar todo o dataset e aplicar lógicas afim de popular os campos ilesos, feridos_leves, feridos_graves e mortos (inseridos de 2017 pra frente) com os dados de 2008 a 2016. Feature Engeneering ou 'Engenharia de características' é o nome dado ao processo de utilizar os dados disponíveis para gerar novas informações afim de complementar o dataset

In [4]:
#Verificando valores nulos/nan em cada coluna
rawdata.isnull().sum()

reg_id                          0
id                              0
pesid                           3
data_inversa                    1
dia_semana                      1
horario                         1
uf                              1
br                            543
km                            543
municipio                       1
causa_acidente                  1
tipo_acidente                   1
classificacao_acidente          3
fase_dia                        2
sentido_via                     1
condicao_metereologica          8
tipo_pista                      1
tracado_via                     1
uso_solo                        1
id_veiculo                   4000
tipo_veiculo                 4003
marca                       34014
ano_fabricacao_veiculo      33707
tipo_envolvido                  1
estado_fisico                   4
idade                       70739
sexo                          681
nacionalidade              363588
naturalidade               494005
ilesos        

In [5]:
#Salvando o número de linhas atual do dataset
antes = rawdata.shape[0]

#Dropando as colunas que não serão utilizadas
#OBS: nacionalidade e naturalidade seriam utilizadas mais adiante, mas visto o numero de registros em branco, acabei optando por descartar
rawdata.drop(labels= ['nacionalidade','naturalidade','regional','delegacia','uop'], axis = 1, inplace = True)

#Dropando os registros que tem algum valor em branco nas colunas presentes no parâmetro subset
rawdata.dropna(subset= ['pesid','data_inversa','dia_semana','horario','uf','br','km','municipio','causa_acidente','tipo_acidente','classificacao_acidente',
                       'fase_dia','sentido_via','condicao_metereologica','tipo_pista','tracado_via','uso_solo','id_veiculo','tipo_veiculo','marca',
                       'ano_fabricacao_veiculo','tipo_envolvido','estado_fisico','sexo'], how = 'any', axis = 0, inplace = True)

print('Tamanho anterior do dataset em linhas: {} \nTotal de registros removidos: {}'.format(antes,(antes - rawdata.shape[0])))
print('Tamanho atual do dataset em linhas: {}'.format(rawdata.shape[0]))

Tamanho anterior do dataset em linhas: 3453255 
Total de registros removidos: 54924
Tamanho atual do dataset em linhas: 3398331


In [6]:
#Verificando novamente os valores nulos/nan em cada coluna
rawdata.isnull().sum()

reg_id                          0
id                              0
pesid                           0
data_inversa                    0
dia_semana                      0
horario                         0
uf                              0
br                              0
km                              0
municipio                       0
causa_acidente                  0
tipo_acidente                   0
classificacao_acidente          0
fase_dia                        0
sentido_via                     0
condicao_metereologica          0
tipo_pista                      0
tracado_via                     0
uso_solo                        0
id_veiculo                      0
tipo_veiculo                    0
marca                           0
ano_fabricacao_veiculo          0
tipo_envolvido                  0
estado_fisico                   0
idade                       45030
sexo                            0
ilesos                    3064289
feridos_leves             3064289
feridos_graves

Beleza, eliminamos todos os registros em branco exceto a idade, embora o número de registros com a idade em branco tenha caído, não deletei estes registros pois existem algumas formas de preencher estes valores faltantes sem alterar as estatístcas da massa de dados. E os outros 6 campos restantes, serão populados mais tarde

#### Tratando os dados referentes às vítimas
Durante o processo de tratamento do campo idade, reparei que todos os campos com informações referentes às vítimas precisavam de limpeza e tratamento...

In [7]:
#Consolidando os tipos de vítima
rawdata['tipo_envolvido'] = rawdata['tipo_envolvido'].str.rstrip()

In [8]:
#Verificando a contagem de registros para sexo
rawdata['sexo'].value_counts()

Masculino               2297239
Feminino                 478765
Masculino                245039
M                        138503
Inválido                 104197
Feminino                  75738
F                         39731
Não Informado             12602
Não Informado              3147
I                          2707
Ignorado                    663
Name: sexo, dtype: int64

In [9]:
#Normalizando os registros p/ sexo
rawdata['sexo'] = rawdata['sexo'].str.rstrip()
rawdata['sexo'].replace(to_replace= {'M':'Masculino','F':'Feminino','I':'Não Informado','Inválido':'Não Informado','Ignorado':'Não Informado'}, inplace= True)
rawdata['sexo'].value_counts()

Masculino        2680781
Feminino          594234
Não Informado     123316
Name: sexo, dtype: int64

In [10]:
#Verificando a contagem de registros para estado_fisico
rawdata['estado_fisico'].value_counts()

Ileso                   1914969
Ferido Leve              575836
Ferido Grave             228243
Ileso                    166218
Ignorado                 117300
Lesões Leves             114071
Ileso                     99904
Morto                     63742
Ferido Leve               55332
Lesões Graves             31986
Não Informado             11940
Óbito                      9827
Morto                      4551
Ignorado                   4412
Name: estado_fisico, dtype: int64

In [11]:
#Normalizando os registros p/ estado_fisico
rawdata['estado_fisico'] = rawdata['estado_fisico'].str.rstrip()
rawdata['estado_fisico'].value_counts()

Ileso            2181091
Ferido Leve       631168
Ferido Grave      228243
Ignorado          121712
Lesões Leves      114071
Morto              68293
Lesões Graves      31986
Não Informado      11940
Óbito               9827
Name: estado_fisico, dtype: int64

#### Tratando o campo idade
Com as informações referente às vítimas organizadas, fica mais fácil a filtragem e tratamento do campo idade

In [12]:
#Observando estatísticas básicas sobre o atributo idade, com saída convertida em int
rawdata['idade'].describe().astype(int)

count    3353301
mean          33
std           29
min           -1
25%           24
50%           33
75%           45
max         2017
Name: idade, dtype: int32

Observando as estatísticas básicas acima, fica claro que temos idades com problemas, poderia remover os registros problemáticos, ou ainda alterar seu valor para a média de 33 anos o que teria um menor impacto. Mas decidi fazer o segunte: Se for o registro de um condutor e a idade for <= 0, deleta-se o registro, já que bebês não dirigem fica claro ser um registro que não tem a consistência desejada

In [13]:
#Verificando registros que atendem ao filtro
rawdata[(rawdata['idade'] <= 0) & (rawdata['tipo_envolvido'] == 'Condutor')]

,reg_id,id,pesid,data_inversa,dia_semana,horario,uf,br,km,municipio,...,tipo_envolvido,estado_fisico,idade,sexo,ilesos,feridos_leves,feridos_graves,mortos,latitude,longitude
3,166203,408531,1178349.0,20/07/2008,Domingo,23:00:00,SP,116,44.5,CANAS,...,Condutor,Ileso,-1.0,Masculino,NaN,NaN,NaN,NaN,NaN,NaN
4,166204,408531,1178362.0,20/07/2008,Domingo,23:00:00,SP,116,44.5,CANAS,...,Condutor,Ferido Leve,-1.0,Masculino,NaN,NaN,NaN,NaN,NaN,NaN
41,166241,408551,1197734.0,02/08/2008,Sábado,19:30:00,PE,232,69,GRAVATA,...,Condutor,Ignorado,-1.0,Não Informado,NaN,NaN,NaN,NaN,NaN,NaN
108,166308,408582,1177921.0,31/07/2008,Quinta,15:00:00,PB,230,153.5,CAMPINA GRANDE,...,Condutor,Ignorado,-1.0,Não Informado,NaN,NaN,NaN,NaN,NaN,NaN
112,166312,408583,1177896.0,26/07/2008,Sábado,20:00:00,RR,174,361,CARACARAI,...,Condutor,Ignorado,-1.0,Não Informado,NaN,NaN,NaN,NaN,NaN,NaN
114,166314,408584,1177902.0,28/07/2008,Segunda,12:00:00,SC,280,72.6,JARAGUA DO SUL,...,Condutor,Ileso,-1.0,Masculino,NaN,NaN,NaN,NaN,NaN,NaN
167,166367,408614,1178021.0,03/08/2008,Domingo,07:25:00,SP,116,269.7,TABOAO DA SERRA,...,Condutor,Ileso,-1.0,Feminino,NaN,NaN,NaN,NaN,NaN,NaN
173,166373,408616,1177983.0,03/08/2008,Domingo,11:50:00,MG,262,43.7,MANHUACU,...,Condutor,Ignorado,-1.0,Não Informado,NaN,NaN,NaN,NaN,NaN,NaN
186,166386,408624,1178494.0,03/08/2008,Domingo,10:30:00,SP,381,74,SAO PAULO,...,Condutor,Ignorado,-1.0,Não Informado,NaN,NaN,NaN,NaN,NaN,NaN
191,166391,408627,1178036.0,28/07/2008,Segunda,19:00:00,ES,262,120,CONCEICAO DO CASTELO,...,Condutor,Ileso,-1.0,Masculino,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
#Armazenando os indexes das linhas que atendem ao filtro em uma mascara e dropando as linhas
f = rawdata[(rawdata['idade'] <= 0) & (rawdata['tipo_envolvido'] == 'Condutor')].index
rawdata.drop(f,axis=0, inplace= True)

In [15]:
#Verificando novamente as estatísticas básicas da massa de dados após a exclusão dos registros
rawdata['idade'].describe().astype(int)

count    3109586
mean          35
std           28
min           -1
25%           26
50%           35
75%           46
max         2017
Name: idade, dtype: int32

In [16]:
#Verificando a quantidade de valores null
rawdata['idade'].isnull().value_counts()

False    3109586
True       45030
Name: idade, dtype: int64

In [17]:
#Verificando tipo de víima envolvida de acordo com a idade
rawdata[rawdata['idade'] <= 0]['tipo_envolvido'].value_counts()

Passageiro               143608
Pedestre                  10579
Cavaleiro                   150
Proprietário de Carga         1
Name: tipo_envolvido, dtype: int64

Após testes em uma cópia dos dados originais, levando em consideração que a idéia é deletar o menor número possível de registros com a idade em branco, e gerar o menor impacto nas métricas da massa de dados, adotei as seguintes medidas conforme bloco de comandos abaixo:

In [18]:
#Os valores null's do dataset, recebem como idade o valor da média das idades (mesmo valor da mediana no caso)
rawdata['idade'].fillna(value= 35, inplace= True)

#A variável f recebe o index ou posição de todos os registros em que a idade é maior do que 100
f = rawdata[rawdata['idade'] >= 100].index
#E em seguida, é utilizada como máscara para deletar todos os registros selecionados
rawdata.drop(f,axis = 0, inplace = True)

#Em seguida, todos os registros com a flag -1 no campo idade, tem a mesma substituída por um valor null
rawdata['idade'].replace(-1,np.nan, inplace= True)

# Que é interpolado visando redistribuir os mesmos no range de idade de forma o mais natural possível
rawdata['idade'].interpolate(inplace= True)

Realizado o tratamento do campo idade, posso verificar que outras colunas do dataset precisam de tratamento, começando com as variáveis categóricas, como fase_dia, traçado_pista, etc.

#### Tratando as outras colunas do dataset
Depois de ter ajustado as que faziam referência às vítimas, segui este processo nos 2 blocos de comando abaixo para o restante do dataset. Onde apenas os 2 primeiros e os que necessitaram de atenção especial estão dispostos por mais de um bloco, e o restante por seguir o mesmo processo dos 2 blocos abaixo, foram 'abreviados' em função do comprimento do documento

In [19]:
#Verificando os valores para a coluna dia da semana
rawdata['dia_semana'].value_counts()

Sexta            432786
Domingo          431020
Sábado           410283
Segunda          387817
Quinta           357279
Quarta           343523
Terça            331701
domingo           56837
sábado            55792
sexta-feira       51722
segunda-feira     44864
quinta-feira      43316
quarta-feira      40681
terça-feira       39632
Sábado            30629
Sexta             28551
Quinta            23528
Quarta            22194
Terça             21220
Name: dia_semana, dtype: int64

In [20]:
#Removendo os espaços à direita
rawdata['dia_semana'] = rawdata['dia_semana'].str.rstrip()

#Consolidando os registros de dia da semana
rawdata['dia_semana'].replace(to_replace = {'domingo':'Domingo','segunda-feira':'Segunda','terça-feira':'Terça','quarta-feira':'Quarta',
                                            'quinta-feira':'Quinta','sexta-feira':'Sexta','sábado':'Sábado'}, inplace= True)

#Checagem dos valores
rawdata['dia_semana'].value_counts()

Sexta      513059
Sábado     496704
Domingo    487857
Segunda    432681
Quinta     424123
Quarta     406398
Terça      392553
Name: dia_semana, dtype: int64

In [21]:
#Verificando os valores para a coluna UF
#rawdata['uf'].value_counts()

#Verificando os registros que possuem (null) no valor de UF
rawdata.loc[rawdata.uf == '(null)']

,reg_id,id,pesid,data_inversa,dia_semana,horario,uf,br,km,municipio,...,tipo_envolvido,estado_fisico,idade,sexo,ilesos,feridos_leves,feridos_graves,mortos,latitude,longitude
56141,132476,392093,1123381.0,20/06/2008,Sexta,17:40:00,(null),(null),(null),PEIXOTO DE AZEVEDO,...,Passageiro,Ferido Leve,54.0,Feminino,NaN,NaN,NaN,NaN,NaN,NaN
56142,132477,392093,1123392.0,20/06/2008,Sexta,17:40:00,(null),(null),(null),PEIXOTO DE AZEVEDO,...,Condutor,Ferido Grave,68.0,Masculino,NaN,NaN,NaN,NaN,NaN,NaN
143056,93421,372626,1059884.0,01/05/2008,Quinta,18:00:00,(null),(null),(null),PEIXOTO DE AZEVEDO,...,Condutor,Ferido Grave,20.0,Masculino,NaN,NaN,NaN,NaN,NaN,NaN
161707,46009,348761,981634.0,01/03/2008,Sábado,01:30:00,(null),(null),(null),PEIXOTO DE AZEVEDO,...,Condutor,Ferido Leve,43.0,Masculino,NaN,NaN,NaN,NaN,NaN,NaN
161708,46010,348761,981645.0,01/03/2008,Sábado,01:30:00,(null),(null),(null),PEIXOTO DE AZEVEDO,...,Passageiro,Ferido Grave,26.0,Feminino,NaN,NaN,NaN,NaN,NaN,NaN
161709,46011,348761,981681.0,01/03/2008,Sábado,01:30:00,(null),(null),(null),PEIXOTO DE AZEVEDO,...,Passageiro,Ferido Grave,25.0,Feminino,NaN,NaN,NaN,NaN,NaN,NaN
161710,46012,348761,981689.0,01/03/2008,Sábado,01:30:00,(null),(null),(null),PEIXOTO DE AZEVEDO,...,Passageiro,Ferido Leve,23.0,Feminino,NaN,NaN,NaN,NaN,NaN,NaN
161711,46013,348761,981697.0,01/03/2008,Sábado,01:30:00,(null),(null),(null),PEIXOTO DE AZEVEDO,...,Passageiro,Ileso,21.0,Feminino,NaN,NaN,NaN,NaN,NaN,NaN
1523625,1596817,1112105,3434053.0,10/06/2012,Domingo,19:11:00,(null),(null),(null),SAO JOAO DE MERITI,...,Condutor,Ferido Leve,35.0,Masculino,NaN,NaN,NaN,NaN,NaN,NaN
1532846,1510623,1069183,3295929.0,03/03/2012,Sábado,19:45:00,(null),(null),(null),GUARANTA DO NORTE,...,Condutor,Ileso,26.0,Masculino,NaN,NaN,NaN,NaN,NaN,NaN


Estes registros poderiam ser aproveitados, pois tem as informações do estado físico das vítimas, e o município onde ocorreu o acidente. Mas como são poucos registros e nenhum possui a informação referente a UF, BR (estrada) e KM do evento, serão descartados.

In [22]:
f = rawdata.loc[rawdata.uf == '(null)'].index
rawdata.drop(f, axis= 0, inplace = True)

In [23]:
#Verificando os valores para a coluna causa_acidente
#rawdata['causa_acidente'].value_counts()

#Substituindo a causa null pela causa 'Outras' já presente no dataset
rawdata['causa_acidente'].replace('(null)','Outras', inplace= True)

#Removendo os espaços à direita dos registros
rawdata['causa_acidente'] = rawdata['causa_acidente'].str.rstrip()

#Consolidando causas iguais
rawdata['causa_acidente'].replace(to_replace= {'Falta de atenção':'Falta de Atenção à Condução','Ingestão de álcool':'Ingestão de Álcool',
                                              'Defeito mecânico em veículo':'Defeito Mecânico no Veículo','Dormindo':'Condutor Dormindo',
                                               'Ultrapassagem indevida':'Ultrapassagem Indevida'}, inplace = True)
#Verificando alterações feitas
rawdata['causa_acidente'].value_counts()

Falta de Atenção à Condução                                                       1096709
Outras                                                                             721525
Não guardar distância de segurança                                                 333018
Velocidade incompatível                                                            223261
Ingestão de Álcool                                                                 140892
Desobediência à sinalização                                                        134891
Defeito Mecânico no Veículo                                                        120269
Ultrapassagem Indevida                                                              93555
Condutor Dormindo                                                                   80582
Animais na Pista                                                                    62238
Defeito na via                                                                      37043
Velocidade

In [24]:
#Verficando os valores para a coluna tipo_acidente
#rawdata['tipo_acidente'].value_counts()

#Removendo os espaços à direita dos registros
rawdata['tipo_acidente'] = rawdata['tipo_acidente'].str.rstrip()

#Consolidando registros de mesmo tipo
rawdata['tipo_acidente'].replace(to_replace= {'Colisão Transversal':'Colisão transversal', 'Atropelamento de pessoa':'Atropelamento de Pedestre',
                                              'Atropelamento de animal':'Atropelamento de Animal', 'Derramamento de carga':'Derramamento de Carga',
                                              'Queda de motocicleta / bicicleta / veículo':'Queda de ocupante de veículo','Danos eventuais':'Danos Eventuais'}, inplace = True)
#Verificando as alterações realizadas
rawdata['tipo_acidente'].value_counts()

Colisão traseira                   1020507
Colisão lateral                     540452
Colisão transversal                 380193
Saída de Pista                      309007
Colisão frontal                     200902
Capotamento                         129452
Colisão com objeto fixo             111032
Tombamento                           91586
Atropelamento de Pedestre            86620
Queda de ocupante de veículo         70809
Atropelamento de Animal              50228
Saída de leito carroçável            44970
Colisão com bicicleta                27388
Colisão com objeto móvel             23905
Colisão com objeto estático          19068
Danos Eventuais                      13300
Incêndio                             11693
Derramamento de Carga                10512
Engavetamento                         9765
Colisão com objeto em movimento       1970
Name: tipo_acidente, dtype: int64

In [25]:
#Removendo espaços à direita dos registros da coluna classificação_acidente
rawdata['classificacao_acidente'] = rawdata['classificacao_acidente'].str.rstrip()

#Verificação dos registros que tem a classificação do acidente como (null)
#Utilização de opção de contexto do Pandas, para que o número maximo de colunas exibidas no dataset seja maior que o número de colunas do mesmo
with pd.option_context("display.max_columns", 40):
    display(rawdata.loc[rawdata['classificacao_acidente'] == '(null)'])

#Mais pra frente a classificação destes registros será corrigida - OK DONE

,reg_id,id,pesid,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,classificacao_acidente,fase_dia,sentido_via,condicao_metereologica,tipo_pista,tracado_via,uso_solo,id_veiculo,tipo_veiculo,marca,ano_fabricacao_veiculo,tipo_envolvido,estado_fisico,idade,sexo,ilesos,feridos_leves,feridos_graves,mortos,latitude,longitude
977939,1031445,26946,2715647.0,16/04/2011,Sábado,22:20:00,SP,153,59.9,SAO JOSE DO RIO PRETO,Ingestão de Álcool,Saída de Pista,(null),Plena noite,Crescente,Ceu Claro,Dupla,Reta,Urbano,2502155,Automóvel,GM/ZAFIRA CD,2002,Condutor,Ileso,40.0,Masculino,NaN,NaN,NaN,NaN,NaN,NaN
2308848,2246470,8,5018708.0,22/11/2014,Sábado,09:35:00,PR,376,335.6,ORTIGUEIRA,Defeito Mecânico no Veículo,Saída de Pista,(null),Pleno dia,Crescente,Chuva,Simples,Curva,Rural,18763467,Automóvel,FORD/ESCORT L ...,1987,Condutor,Ileso,34.0,Masculino,NaN,NaN,NaN,NaN,NaN,NaN
2308849,2246471,8,5018708.0,22/11/2014,Sábado,09:35:00,PR,376,335.6,ORTIGUEIRA,Defeito Mecânico no Veículo,Saída de Pista,(null),Pleno dia,Crescente,Ceu Claro,Simples,Curva,Rural,18763467,Automóvel,FORD/ESCORT L ...,1987,Condutor,Ileso,34.0,Masculino,NaN,NaN,NaN,NaN,NaN,NaN
2308850,2246472,8,5018711.0,22/11/2014,Sábado,09:35:00,PR,376,335.6,ORTIGUEIRA,Defeito Mecânico no Veículo,Saída de Pista,(null),Pleno dia,Crescente,Chuva,Simples,Curva,Rural,18763468,Caminhão-Trator,SCANIA/G 380 A6X2,2013,Condutor,Ileso,63.0,Masculino,NaN,NaN,NaN,NaN,NaN,NaN
2308851,2246473,8,5018711.0,22/11/2014,Sábado,09:35:00,PR,376,335.6,ORTIGUEIRA,Defeito Mecânico no Veículo,Saída de Pista,(null),Pleno dia,Crescente,Ceu Claro,Simples,Curva,Rural,18763468,Caminhão-Trator,SCANIA/G 380 A6X2,2013,Condutor,Ileso,63.0,Masculino,NaN,NaN,NaN,NaN,NaN,NaN
2308854,2246476,9,5019034.0,17/11/2014,Segunda,20:50:00,BA,324,530,FEIRA DE SANTANA,Ultrapassagem Indevida,Colisão frontal,(null),Plena noite,Crescente,Chuva,Dupla,Curva,Urbano,18763578,Caminhonete,NISSAN/FRONTIER,2014,Condutor,Ileso,32.0,Masculino,NaN,NaN,NaN,NaN,NaN,NaN
2308856,2246478,13,5018752.0,20/11/2014,Quinta,09:30:00,BA,367,31.6,PORTO SEGURO,Outras,Saída de Pista,(null),Pleno dia,Crescente,Nublado,Simples,Reta,Rural,18763483,Automóvel,I/FIAT SIENA ELX,2001,Condutor,Ferido Leve,35.0,Masculino,NaN,NaN,NaN,NaN,NaN,NaN
2308857,2246479,14,5018864.0,09/11/2014,Domingo,22:50:00,SC,101,220.9,PALHOCA,Falta de Atenção à Condução,Colisão com objeto fixo,(null),Plena noite,Decrescente,Ceu Claro,Dupla,Reta,Urbano,18763525,Automóvel,I/HYUNDAI AZERA 3.0 V6,2011,Condutor,Ferido Leve,35.0,Masculino,NaN,NaN,NaN,NaN,NaN,NaN
2308858,2246480,15,5018886.0,19/11/2014,Quarta,05:00:00,RJ,116,211,SEROPEDICA,Outras,Atropelamento de Pedestre,(null),Amanhecer,Crescente,Ceu Claro,Dupla,Reta,Urbano,18763527,Automóvel,FORD/FIESTA SEDAN1.6FLEX,2013,Condutor,Ileso,29.0,Masculino,NaN,NaN,NaN,NaN,NaN,NaN
2308859,2246481,16,5018712.0,22/11/2014,Sábado,07:50:00,PR,277,95,CURITIBA,Defeito Mecânico no Veículo,Saída de Pista,(null),Pleno dia,Decrescente,Chuva,Dupla,Curva,Rural,18763471,Caminhonete,I/FORD RANGER XLT 12D,2001,Condutor,Ferido Leve,32.0,Masculino,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
#Verificação dos regstros para a coluna fase_dia
#rawdata['fase_dia'].value_counts()

#Verificação dos registros com período do dia como (null). Como todos possuem os horários dos acidentes, podem ser corrigidos mais adiante
rawdata.loc[rawdata.fase_dia == '(null)'] #arruma dps

,reg_id,id,pesid,data_inversa,dia_semana,horario,uf,br,km,municipio,...,tipo_envolvido,estado_fisico,idade,sexo,ilesos,feridos_leves,feridos_graves,mortos,latitude,longitude
93659,125779,388803,1112736.0,12/06/2008,Quinta,07:40:00,SC,101,131.5,BALNEARIO CAMBORIU,...,Condutor,Ileso,27.0,Feminino,NaN,NaN,NaN,NaN,NaN,NaN
102365,104026,377980,1077502.0,14/05/2008,Quarta,16:35:00,PA,316,8,ANANINDEUA,...,Condutor,Ileso,64.0,Masculino,NaN,NaN,NaN,NaN,NaN,NaN
102366,104027,377980,1077510.0,14/05/2008,Quarta,16:35:00,PA,316,8,ANANINDEUA,...,Condutor,Ileso,24.0,Masculino,NaN,NaN,NaN,NaN,NaN,NaN
155968,128572,390159,1116746.0,15/06/2008,Domingo,23:00:00,RJ,40,39,AREAL,...,Passageiro,Ferido Grave,23.0,Masculino,NaN,NaN,NaN,NaN,NaN,NaN
180045,144978,398215,1143569.0,05/07/2008,Sábado,10:40:00,GO,40,73,CRISTALINA,...,Condutor,Ileso,54.0,Masculino,NaN,NaN,NaN,NaN,NaN,NaN
180046,144979,398215,1143598.0,05/07/2008,Sábado,10:40:00,GO,40,73,CRISTALINA,...,Condutor,Ileso,35.0,Masculino,NaN,NaN,NaN,NaN,NaN,NaN
243675,193133,421933,1220890.0,02/09/2008,Terça,21:20:00,RJ,101,263.5,RIO BONITO,...,Condutor,Ileso,40.0,Masculino,NaN,NaN,NaN,NaN,NaN,NaN
243676,193134,421933,1220893.0,02/09/2008,Terça,21:20:00,RJ,101,263.5,RIO BONITO,...,Pedestre,Ferido Leve,58.0,Masculino,NaN,NaN,NaN,NaN,NaN,NaN
248306,344816,494955,1455392.0,25/02/2009,Quarta,16:40:00,MG,116,413.6,GOVERNADOR VALADARES,...,Condutor,Ileso,29.0,Masculino,NaN,NaN,NaN,NaN,NaN,NaN
248307,344817,494955,1455402.0,25/02/2009,Quarta,16:40:00,MG,116,413.6,GOVERNADOR VALADARES,...,Condutor,Ileso,26.0,Masculino,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
#Remoção dos espaços à direita dos registros
rawdata['fase_dia'] = rawdata['fase_dia'].str.rstrip()
rawdata['fase_dia'].value_counts()

Pleno dia      1871962
Plena noite     824357
Anoitecer       194030
Amanhecer       154306
Plena Noite     108615
(null)              89
Name: fase_dia, dtype: int64

In [28]:
#Verificação dos regstros para a coluna sentido_via
#rawdata['sentido_via'].value_counts()

#Remoção dos espaços à direita dos registros
rawdata['sentido_via'] = rawdata['sentido_via'].str.rstrip()
#Verificação das alterações realizadas
rawdata['sentido_via'].value_counts()

Crescente      1625808
Decrescente    1527551
Name: sentido_via, dtype: int64

In [29]:
#Verificação dos regstros para a coluna condicao_meteorologica
#rawdata['condicao_metereologica'].value_counts()

#Remoção dos espaços à direita dos registros
rawdata['condicao_metereologica'] = rawdata['condicao_metereologica'].str.rstrip()

#Consolidação dos valores dos registros
rawdata['condicao_metereologica'].replace(to_replace= {'Céu Claro':'Ceu Claro','Ignorado':'Ignorada', 'Nevoeiro/neblina':'Nevoeiro/Neblina', '(null)':'Ignorada'}, inplace = True)

#Verificação das alterações realizadas
rawdata['condicao_metereologica'].value_counts()

Ceu Claro           1556502
Nublado              570631
Chuva                507512
Sol                  414327
Ignorada              52110
Nevoeiro/Neblina      32132
Garoa/Chuvisco        11484
Vento                  7995
Granizo                 468
Neve                    198
Name: condicao_metereologica, dtype: int64

In [30]:
#Verificação dos regstros para a coluna tipo_pista
#rawdata['tipo_pista'].value_counts()

#Verificação dos regstros para a coluna tipo_pista com o valor (null), que pode ser corrigido manualmente por serem poucos registros
rawdata[rawdata.tipo_pista == '(null)'] #arruma dps

,reg_id,id,pesid,data_inversa,dia_semana,horario,uf,br,km,municipio,...,tipo_envolvido,estado_fisico,idade,sexo,ilesos,feridos_leves,feridos_graves,mortos,latitude,longitude
1057829,1086726,858063,2617891.0,17/02/2011,Quinta,03:30:00,MG,381,687.9,LAVRAS,...,Condutor,Ileso,55.0,Masculino,NaN,NaN,NaN,NaN,NaN,NaN
1143906,1114698,871779,2662718.0,12/03/2011,Sábado,04:20:00,CE,116,8,FORTALEZA,...,Condutor,Ferido Leve,29.0,Masculino,NaN,NaN,NaN,NaN,NaN,NaN
1438673,1335773,984377,3017467.0,01/10/2011,Sábado,14:30:00,GO,70,39,COCALZINHO DE GOIAS,...,Condutor,Ferido Leve,33.0,Masculino,NaN,NaN,NaN,NaN,NaN,NaN
1491073,1504702,1066266,3283152.0,23/02/2012,Quinta,11:00:00,MG,40,41,PARACATU,...,Condutor,Ferido Leve,32.0,Masculino,NaN,NaN,NaN,NaN,NaN,NaN
1588674,1516963,1072411,3303061.0,10/03/2012,Sábado,03:30:00,RS,290,87.1,PORTO ALEGRE,...,Condutor,Ileso,23.0,Masculino,NaN,NaN,NaN,NaN,NaN,NaN
2406663,2286089,83143014,4560648.0,03/02/2014,Segunda,17:20:00,PA,316,4,ANANINDEUA,...,Condutor,Ileso,29.0,Masculino,NaN,NaN,NaN,NaN,NaN,NaN
2406664,2286090,83143014,4560664.0,03/02/2014,Segunda,17:20:00,PA,316,4,ANANINDEUA,...,Condutor,Ileso,58.0,Masculino,NaN,NaN,NaN,NaN,NaN,NaN
2451805,2571023,83280429,5023074.0,24/11/2014,Segunda,12:10:00,MG,381,493,BETIM,...,Condutor,Ileso,34.0,Masculino,NaN,NaN,NaN,NaN,NaN,NaN
2578439,2568215,83278982,5018698.0,19/11/2014,Quarta,09:20:00,RJ,493,17,GUAPIMIRIM,...,Condutor,Ileso,42.0,Feminino,NaN,NaN,NaN,NaN,NaN,NaN
2578440,2568216,83278982,5018701.0,19/11/2014,Quarta,09:20:00,RJ,493,17,GUAPIMIRIM,...,Condutor,Ferido Leve,36.0,Masculino,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
#Remoção dos espaços à direita dos registros
rawdata['tipo_pista'] = rawdata['tipo_pista'].str.rstrip()
#Verificação das alterações realizadas
rawdata['tipo_pista'].value_counts()

Simples     1696825
Dupla       1220604
Múltipla     235918
(null)           12
Name: tipo_pista, dtype: int64

In [32]:
#Verificação dos regstros para a coluna tracado_via
#rawdata['tracado_via'].value_counts()

#Remoção dos registros onde o traçado da via tem (null) como valor
f = rawdata[rawdata.tracado_via == '(null)'].index
rawdata.drop(f, axis = 0, inplace = True)

#Remoção dos espaços à direita dos registros
rawdata['tracado_via'] = rawdata['tracado_via'].str.rstrip()

#Verificação das alterações realizadas
rawdata['tracado_via'].value_counts()

Reta                     2286055
Curva                     599797
Cruzamento                193350
Não Informado              33723
Interseção de vias         15987
Desvio Temporário           9672
Rotatória                   6380
Retorno Regulamentado       3815
Viaduto                     2350
Ponte                       1920
Túnel                        298
Name: tracado_via, dtype: int64

In [33]:
#Verificação dos regstros para a coluna uso_solo
#rawdata['uso_solo'].value_counts()

#Remoção dos espaços à direita dos registros
rawdata['uso_solo'] = rawdata['uso_solo'].str.rstrip()

#Consolidação dos registros
rawdata['uso_solo'].replace(to_replace= {'Sim':'Urbano','Não':'Rural'}, inplace = True)

#Verificação das alterações realizadas
rawdata['uso_solo'].value_counts()

Urbano    1639781
Rural     1513566
Name: uso_solo, dtype: int64

In [34]:
#Remoção dos espaços à direita dos registros
rawdata['tipo_veiculo'] = rawdata['tipo_veiculo'].str.rstrip()

#Consolidação dos registros
rawdata['tipo_veiculo'].replace(to_replace= {'Motocicletas':'Motocicleta','Micro-ônibus':'Microônibus','Semi-Reboque':'Semireboque','(null)':'Desconhecido'}, inplace = True)

#Verificação das alterações realizadas
rawdata['tipo_veiculo'].value_counts()

Automóvel             1564107
Motocicleta            337648
Caminhão               294943
Caminhão-Trator        285029
Caminhonete            243986
Ônibus                 132073
Camioneta              104979
Desconhecido            42449
Motoneta                35099
Microônibus             32483
Caminhão-trator         28475
Utilitário              21241
Bicicleta               18932
Ciclomotor               4649
Semireboque              1616
Trator de rodas          1381
Carroça                  1125
Não identificado          986
Caminhão-Tanque           975
Reboque                   342
Triciclo                  251
Charrete                  163
Bonde / Trem              147
Trator misto              117
Trator de esteiras         44
Carro-de-mao               43
Não Informado              22
Outros                     20
Motor-Casa                 11
Quadriciclo                10
Trem-bonde                  1
Name: tipo_veiculo, dtype: int64

In [35]:
#Remoção dos espaços à direita dos registros
rawdata['marca'] = rawdata['marca'].str.rstrip()

#Consolidação dos registros
rawdata['marca'].replace(to_replace= {'I':'Importado-Desconhecido','(null)':'Desconhecido'}, inplace = True)

#### Acertando os detalhes
Em algumas colunas existem registros com o valor '(null)' que marquei para correção manual. Correções estas que vão de classificação de registros, a input de dados levantados manualmente de acordo com pesquisas naquele site (Aquele um das buscas)

##### Classificação do Acidente
Temos 20 registros que não foram classificados de acordo com o estado das vítimas, embora os registros tragam a informação referente ao estado das vítimas. Basta então identificar estes 20 registros, buscar a informação no campo estado_vitima e estabelecer a regra de relacionamento de acordo com os outros registros

In [36]:
with pd.option_context('max_columns',45):
    display(rawdata.loc[rawdata['classificacao_acidente'] == '(null)'])

,reg_id,id,pesid,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,classificacao_acidente,fase_dia,sentido_via,condicao_metereologica,tipo_pista,tracado_via,uso_solo,id_veiculo,tipo_veiculo,marca,ano_fabricacao_veiculo,tipo_envolvido,estado_fisico,idade,sexo,ilesos,feridos_leves,feridos_graves,mortos,latitude,longitude
977939,1031445,26946,2715647.0,16/04/2011,Sábado,22:20:00,SP,153,59.9,SAO JOSE DO RIO PRETO,Ingestão de Álcool,Saída de Pista,(null),Plena noite,Crescente,Ceu Claro,Dupla,Reta,Urbano,2502155,Automóvel,GM/ZAFIRA CD,2002,Condutor,Ileso,40.0,Masculino,NaN,NaN,NaN,NaN,NaN,NaN
2308848,2246470,8,5018708.0,22/11/2014,Sábado,09:35:00,PR,376,335.6,ORTIGUEIRA,Defeito Mecânico no Veículo,Saída de Pista,(null),Pleno dia,Crescente,Chuva,Simples,Curva,Rural,18763467,Automóvel,FORD/ESCORT L,1987,Condutor,Ileso,34.0,Masculino,NaN,NaN,NaN,NaN,NaN,NaN
2308849,2246471,8,5018708.0,22/11/2014,Sábado,09:35:00,PR,376,335.6,ORTIGUEIRA,Defeito Mecânico no Veículo,Saída de Pista,(null),Pleno dia,Crescente,Ceu Claro,Simples,Curva,Rural,18763467,Automóvel,FORD/ESCORT L,1987,Condutor,Ileso,34.0,Masculino,NaN,NaN,NaN,NaN,NaN,NaN
2308850,2246472,8,5018711.0,22/11/2014,Sábado,09:35:00,PR,376,335.6,ORTIGUEIRA,Defeito Mecânico no Veículo,Saída de Pista,(null),Pleno dia,Crescente,Chuva,Simples,Curva,Rural,18763468,Caminhão-Trator,SCANIA/G 380 A6X2,2013,Condutor,Ileso,63.0,Masculino,NaN,NaN,NaN,NaN,NaN,NaN
2308851,2246473,8,5018711.0,22/11/2014,Sábado,09:35:00,PR,376,335.6,ORTIGUEIRA,Defeito Mecânico no Veículo,Saída de Pista,(null),Pleno dia,Crescente,Ceu Claro,Simples,Curva,Rural,18763468,Caminhão-Trator,SCANIA/G 380 A6X2,2013,Condutor,Ileso,63.0,Masculino,NaN,NaN,NaN,NaN,NaN,NaN
2308854,2246476,9,5019034.0,17/11/2014,Segunda,20:50:00,BA,324,530,FEIRA DE SANTANA,Ultrapassagem Indevida,Colisão frontal,(null),Plena noite,Crescente,Chuva,Dupla,Curva,Urbano,18763578,Caminhonete,NISSAN/FRONTIER,2014,Condutor,Ileso,32.0,Masculino,NaN,NaN,NaN,NaN,NaN,NaN
2308856,2246478,13,5018752.0,20/11/2014,Quinta,09:30:00,BA,367,31.6,PORTO SEGURO,Outras,Saída de Pista,(null),Pleno dia,Crescente,Nublado,Simples,Reta,Rural,18763483,Automóvel,I/FIAT SIENA ELX,2001,Condutor,Ferido Leve,35.0,Masculino,NaN,NaN,NaN,NaN,NaN,NaN
2308857,2246479,14,5018864.0,09/11/2014,Domingo,22:50:00,SC,101,220.9,PALHOCA,Falta de Atenção à Condução,Colisão com objeto fixo,(null),Plena noite,Decrescente,Ceu Claro,Dupla,Reta,Urbano,18763525,Automóvel,I/HYUNDAI AZERA 3.0 V6,2011,Condutor,Ferido Leve,35.0,Masculino,NaN,NaN,NaN,NaN,NaN,NaN
2308858,2246480,15,5018886.0,19/11/2014,Quarta,05:00:00,RJ,116,211,SEROPEDICA,Outras,Atropelamento de Pedestre,(null),Amanhecer,Crescente,Ceu Claro,Dupla,Reta,Urbano,18763527,Automóvel,FORD/FIESTA SEDAN1.6FLEX,2013,Condutor,Ileso,29.0,Masculino,NaN,NaN,NaN,NaN,NaN,NaN
2308859,2246481,16,5018712.0,22/11/2014,Sábado,07:50:00,PR,277,95,CURITIBA,Defeito Mecânico no Veículo,Saída de Pista,(null),Pleno dia,Decrescente,Chuva,Dupla,Curva,Rural,18763471,Caminhonete,I/FORD RANGER XLT 12D,2001,Condutor,Ferido Leve,32.0,Masculino,NaN,NaN,NaN,NaN,NaN,NaN


Para o funcionamento desejado do método .iloc() que será utilizado, é necessária a reindexação do dataset, conforme explicado no artigo : [link cheroso]

In [37]:
rawdata.reset_index(inplace= True)

In [42]:
R = rawdata.loc[rawdata['classificacao_acidente'] == '(null)'].index

In [45]:
with pd.option_context('max_columns',45):
    for item in R:
        #print('valor de item: ', item)
        #display(rawdata.iloc[[item]])
        if (rawdata.iloc[[item],[25]].values == 'Ileso'):
            rawdata.iloc[item,13] = 'Sem Vítimas'
            print('Classificação Acidente Atualizada')
            
        elif (rawdata.iloc[[item],[25]].values == 'Ferido Leve'):
            rawdata.iloc[item,13] = 'Com Vítimas Feridas'
            print('Classificação Acidente Atualizada HUE')
            
        elif (rawdata.iloc[[item],[25]].values == 'Morto'):
            rawdata.iloc[item,13] = 'Com Vítimas Fatais'
            print('Classificação Acidente Atualizada R.I.P')
        
        else:
            print('oh noooeZ')


Classificação Acidente Atualizada
Classificação Acidente Atualizada
Classificação Acidente Atualizada
Classificação Acidente Atualizada
Classificação Acidente Atualizada
Classificação Acidente Atualizada
Classificação Acidente Atualizada HUE
Classificação Acidente Atualizada HUE
Classificação Acidente Atualizada
Classificação Acidente Atualizada HUE
Classificação Acidente Atualizada HUE
Classificação Acidente Atualizada HUE
oh noooeZ
Classificação Acidente Atualizada
Classificação Acidente Atualizada R.I.P
Classificação Acidente Atualizada
Classificação Acidente Atualizada
oh noooeZ
Classificação Acidente Atualizada
oh noooeZ


In [47]:
with pd.option_context('max_columns',45):
    display(rawdata.loc[rawdata['classificacao_acidente'] == '(null)'])

,index,reg_id,id,pesid,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,classificacao_acidente,fase_dia,sentido_via,condicao_metereologica,tipo_pista,tracado_via,uso_solo,id_veiculo,tipo_veiculo,marca,ano_fabricacao_veiculo,tipo_envolvido,estado_fisico,idade,sexo,ilesos,feridos_leves,feridos_graves,mortos,latitude,longitude
2120605,2308862,2246484,17,5018910.0,22/11/2014,Sábado,06:10:00,SP,116,550,BARRA DO TURVO,Falta de Atenção à Condução,Colisão traseira,(null),Amanhecer,Crescente,Chuva,Simples,Curva,Rural,18763534,Automóvel,CHEVROLET/CLASSIC LS,2012,Passageiro,Ferido Grave,55.0,Feminino,NaN,NaN,NaN,NaN,NaN,NaN
2120610,2308867,2246489,332,5018917.0,22/11/2014,Sábado,10:20:00,ES,101,146,LINHARES,Desobediência à sinalização,Colisão transversal,(null),Pleno dia,Decrescente,Nublado,Simples,Cruzamento,Urbano,18763512,Automóvel,IMP/FORD ESCORT GL 16V H,1997,Passageiro,Ferido Grave,13.0,Masculino,NaN,NaN,NaN,NaN,NaN,NaN
2120612,2308869,2246491,1175,5019295.0,22/11/2014,Sábado,19:20:00,BA,116,778.1,PLANALTO,Ingestão de Álcool,Atropelamento de Pedestre,(null),Plena noite,Decrescente,Nevoeiro/Neblina,Simples,Reta,Urbano,(null),Desconhecido,Desconhecido,(null),Pedestre,Ferido Grave,31.0,Masculino,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
rawdata.iloc[2120605,13] = 'Com Vítimas Feridas'
rawdata.iloc[2120610,13] = 'Com Vítimas Feridas'
rawdata.iloc[2120612,13] = 'Com Vítimas Fatais'

In [52]:
rawdata.iloc[R,13]

908320             Sem Vítimas
2120594            Sem Vítimas
2120595            Sem Vítimas
2120596            Sem Vítimas
2120597            Sem Vítimas
2120598            Sem Vítimas
2120599    Com Vítimas Feridas
2120600    Com Vítimas Feridas
2120601            Sem Vítimas
2120602    Com Vítimas Feridas
2120603    Com Vítimas Feridas
2120604    Com Vítimas Feridas
2120605    Com Vítimas Feridas
2120606            Sem Vítimas
2120607     Com Vítimas Fatais
2120608            Sem Vítimas
2120609            Sem Vítimas
2120610    Com Vítimas Feridas
2120611            Sem Vítimas
2120612     Com Vítimas Fatais
Name: classificacao_acidente, dtype: object

In [ ]:
#Correção fase do dia
#rawdata.loc[(rawdata.fase_dia == 'Pleno dia'),'horario'].value_counts()

In [ ]:
with pd.option_context('max_columns',45):
    display(rawdata.head())

In [ ]:
rawdata['data_inversa'] = pd.to_datetime(rawdata['data_inversa'])

In [ ]:
test = rawdata.copy()

In [ ]:
rawdata.info()